# Ticker Price Prediction
This notebook focuses on training a machine learning model to predict the future trend of a particular US stock symbol.

We will be using the [yfinance](https://github.com/ranaroussi/yfinance) package to download financial data for US stock symbols

### Packages
Please ensure these packages are installed on your local environment via ```pip install -r requirements.txt``` or the corresponding package manager on your OS.

In [176]:
import csv
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf

%matplotlib inline

### 1. Prerequisites & helper functions

To get started, we will define several functions for retrieving data from the Yahoo Finance API via the yfinance package.

In [265]:
def get_ticker_financial_data(ticker_obj):
    """
    Builds a pandas dataframe containing the essential financial data of a US stock symbol.
    @params ticker_obj: a yfinance Ticker object that provides access to data of a specific ticker
    @return: pandas Dataframe

    """
    last_balance_sheet  = ticker_obj.balance_sheet.iloc[:, 0]
    last_cash_flow      = ticker_obj.cashflow.iloc[:, 0]
    last_earnings       = ticker_obj.earnings.iloc[-1, :]
    data = pd.concat([last_balance_sheet, last_cash_flow, last_earnings])
    return pd.DataFrame(data)

### 2. Get financial data for a specific symbol

In [174]:
ticker = 'AAPL'
yf_ticker = yf.Ticker(ticker)

In [266]:
get_ticker_financial_data(yf_ticker)

,0
Total Liab,3.020830e+11
Total Stockholder Equity,5.067200e+10
Other Current Liab,6.709400e+10
Total Assets,3.527550e+11
Common Stock,6.484900e+10
Other Current Assets,2.122300e+10
Retained Earnings,-3.068000e+09
Other Liab,3.839400e+10
Gains Losses Not Affecting Retained Earnings,-1.110900e+10
Other Assets,4.401100e+10
